In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import spectrogram, stft, istft, check_NOLA
import lightgbm as lgb
from lightgbm import LGBMRegressor

import ray
ray.init(include_dashboard=True, num_cpus = 8, dashboard_host='0.0.0.0')

plt.style.use('ggplot')

2023-03-26 14:28:47,296	INFO worker.py:1553 -- Started a local Ray instance.


# UTILITY FUNCTIONS

In [2]:
subsampleFreq = 64   # FINAL FREQUENCY IN HERTZ AFTER SUBSAMPLING
secondsInWindow = 1.
nperseg = subsampleFreq * secondsInWindow
noverlap = nperseg - 1
window = ('tukey', .25)

In [3]:
# CONVERT STFT FROM R,THETA TO COMPLEX
# dim(z) = (# timesteps, # freq bins x 2 (2 reals = 1 complex))

def rThetaToComplex(z):
    rows, cols = z.shape
    shortTermFourier = np.zeros((rows, cols // 2), dtype=np.csingle)
    for i in range(rows):
        for k in range(cols // 2):
            r = z[i,k]
            theta = z[i, (k + cols // 2)]
            shortTermFourier[i,k] =  r * np.exp(complex(0, theta))
    return shortTermFourier.transpose() # dim = (# freq bins, # timepoints)

# CONVERT REAL STFT TO COMPLEX STFT, INVERT TO GET THE ISTFT (I.E. TIME SERIES), THEN PLOT

def realSTFTtoTimeSeries(realSTFT):
    shortTermFourierComplex = rThetaToComplex(realSTFT)
    times, inverseShortFourier = istft(shortTermFourierComplex, 
                                       fs=subsampleFreq, 
                                       window=window, 
                                       nperseg=nperseg, 
                                       noverlap=noverlap)
    return times, inverseShortFourier

# LOAD NUMPY ARRAYS

In [4]:
arraySavePath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/trainTestRTheta.npz'
npzfile = np.load(arraySavePath)
x_trainRTheta = npzfile['x_trainRTheta']
x_validRTheta = npzfile['x_validRTheta'] 
y_trainRTheta = npzfile['y_trainRTheta'] 
y_validRTheta = npzfile['y_validRTheta']

_,nY = y_validRTheta.shape

# LIGHTGBM

In [5]:
# NATIVE LIGHTGBM API VERSION

modelDirectory = '/blue/gkalamangalam/jmark.ettinger/predictScalp/lgbModels/'

@ray.remote
def train_index(x_trainRTheta, y_trainRTheta, x_validRTheta, y_validRTheta, index, init_model, num_iterations):
    metric = 'mse'
    early_stopping_rounds = 5

    param = {'metric': metric, 
             'num_iterations': num_iterations,
             'early_stopping_rounds': early_stopping_rounds,
             'first_metric_only': True}

    lgbTrain = lgb.Dataset(x_trainRTheta, label=y_trainRTheta[:,index], free_raw_data=False)
    lgbValid = lgbTrain.create_valid(x_validRTheta, label=y_validRTheta[:,index])

    bst = lgb.train(param, 
                    lgbTrain, 
                    valid_sets=[lgbValid, lgbTrain],
                    init_model=init_model)
    
    bst.save_model(modelDirectory + 'lgbModel_%s.txt' % str(index), num_iteration=bst.best_iteration)
    return bst

def lgbPredictIndex(x, index):
    bst = lgb.Booster(model_file=modelDirectory + 'lgbModel_%s.txt' % str(index))
    ypred = bst.predict(x, num_iteration=bst.best_iteration)
    return ypred

def lgbPredict(x):
    return np.array([lgbPredictIndex(x, index) for index in range(nY)]).transpose()

xTrainRay = ray.put(x_trainRTheta)
yTrainRay = ray.put(y_trainRTheta) 
xValidRay = ray.put(x_validRTheta)
yValidRay = ray.put(y_validRTheta)

In [6]:
# TRAIN LIGHTGBM MODEL

continueFlag = True
num_iterations = 50

result_ids = []
for index in range(nY):
    if continueFlag:
        init_model = modelDirectory + 'lgbModel_%s.txt' % str(index)
    else:
        init_model = None
        
    result_ids.append(train_index.remote(xTrainRay, 
                                         yTrainRay, 
                                         xValidRay, 
                                         yValidRay, 
                                         index, 
                                         init_model, 
                                         num_iterations))
#results = ray.get(result_ids)

(train_index pid=11088) /home/jmark.ettinger/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
(train_index pid=11088)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
(train_index pid=11094) /home/jmark.ettinger/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
(train_index pid=11094)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
(train_index pid=11091) /home/jmark.ettinger/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
(train_index pid=11091)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
(train_index pid=11093) /home/jmark.ettinger/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in pa

(train_index pid=11089) [LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 21.707213 seconds.
(train_index pid=11089) You can set `force_col_wise=true` to remove the overhead.
(train_index pid=11089) [LightGBM] [Info] Total Bins 1420188
(train_index pid=11089) [LightGBM] [Info] Number of data points in the train set: 89440, number of used features: 5742
(train_index pid=11089) [123]	training's l2: 0.869031	valid_0's l2: 1.49517
(train_index pid=11089) Training until validation scores don't improve for 5 rounds
(train_index pid=11089) [124]	training's l2: 0.864248	valid_0's l2: 1.49324
(train_index pid=11089) [125]	training's l2: 0.859888	valid_0's l2: 1.49094
(train_index pid=11089) [126]	training's l2: 0.855595	valid_0's l2: 1.48887
(train_index pid=11089) [127]	training's l2: 0.850603	valid_0's l2: 1.48765
(train_index pid=11089) [128]	training's l2: 0.845403	valid_0's l2: 1.48229
(train_index pid=11089) [129]	training's l2: 0.840988	valid_0's l2

(train_index pid=11094) [141]	training's l2: 4.44166	valid_0's l2: 9.98233
(train_index pid=11094) [142]	training's l2: 4.41486	valid_0's l2: 9.97302
(train_index pid=11094) [143]	training's l2: 4.38915	valid_0's l2: 9.95043
(train_index pid=11093) [123]	training's l2: 0.643763	valid_0's l2: 1.23309
(train_index pid=11093) Training until validation scores don't improve for 5 rounds
(train_index pid=11094) [144]	training's l2: 4.36059	valid_0's l2: 9.9324
(train_index pid=11093) [124]	training's l2: 0.640163	valid_0's l2: 1.23115
(train_index pid=11094) [145]	training's l2: 4.33738	valid_0's l2: 9.91886
(train_index pid=11093) [125]	training's l2: 0.636347	valid_0's l2: 1.22948
(train_index pid=11094) [146]	training's l2: 4.31172	valid_0's l2: 9.90747
(train_index pid=11093) [126]	training's l2: 0.632379	valid_0's l2: 1.22695
(train_index pid=11094) [147]	training's l2: 4.28275	valid_0's l2: 9.88912


# Plot results of fit

In [ ]:
# PLOT PREDICTION VERSUS TRUTH

trainPlotFlag = False
    
if trainPlotFlag:
    x = x_trainRTheta
    y = y_trainRTheta
    trainTitle = 'train'
else:
    x = x_validRTheta
    y = y_validRTheta
    trainTitle = 'valididation'
    
freqPredict = lgbPredict(x)
title = 'LightGBM: ' + trainTitle

_, yPred = realSTFTtoTimeSeries(freqPredict)
_, yTrue = realSTFTtoTimeSeries(y)

plt.figure()
plt.plot(yPred, label='predict')
plt.plot(yTrue, label='true')
plt.legend()
plt.title(title)
plt.show()